In [64]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

In [65]:
train_dir = 'images/train'
test_dir = 'images/test'

***CREATING DATASET FOR TRAIN from IMAGEs***<br>
Making a function that takes images from the folder and converts it into a dataframe

In [66]:
def create_dataframe(dir):
    images_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            images_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return images_paths,labels

In [67]:
#converting into dataframe
train = pd.DataFrame()
#making two cloumns
train['image'], train['label'] = create_dataframe(train_dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [68]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


***CREATING DATASET FOR TESTING***

In [69]:
#converting into dataframe
test = pd.DataFrame()
#making two cloumns
test['image'], test['label'] = create_dataframe(train_dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [70]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, target_size=(48,48))
        img = img.convert('L')  # Convert image to grayscale
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [71]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [72]:
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [73]:
#NORMALIZE
x_train = train_features/255.0
x_test = train_features/255.0

In [74]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [75]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [76]:
y_train = to_categorical(y_train, num_classes=7) # we have 7 different emotions
y_test = to_categorical(y_test, num_classes=7)

# Creating a CNN Model

In [77]:
model = Sequential()

#convolutional layers
model.add(Conv2D(128,kernel_size=(3,3), activation='relu', input_shape = (48,48,1) ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
#fully connected layers
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(7, activation='softmax'))

c:\Users\ashok\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [81]:
model.fit(x= x_train, y=y_train, batch_size=256, epochs =10, validation_data=(x_test,y_test))

Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 233s 2s/step - accuracy: 0.2460 - loss: 1.8147 - val_accuracy: 0.2490 - val_loss: 1.7973
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.2502 - loss: 1.8047 - val_accuracy: 0.2575 - val_loss: 1.7763
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.2643 - loss: 1.7634 - val_accuracy: 0.3359 - val_loss: 1.6433
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.3248 - loss: 1.6762 - val_accuracy: 0.4063 - val_loss: 1.5081
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.3891 - loss: 1.5513 - val_accuracy: 0.4679 - val_loss: 1.3712
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.4376 - loss: 1.4568 - val_accuracy: 0.4959 - val_loss: 1.3125
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.4593 - loss: 1.4005 - val_accuracy: 0.5162 - val_loss: 1.2748
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.4749 - loss: 1.3595 - val_accu

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")